In [1]:
import pandas as pd
import glob
import os

def merge_csv_files(folder):
    """
    1) Находит все CSV‑файлы в папке.
    2) Считывает их в DataFrame, очищая пробелы в названиях столбцов.
    3) Объединяет в один общий DataFrame.
    """
    all_files = glob.glob(os.path.join(folder, "*.csv"))
    df_list = []
    for file in all_files:
        try:
            df = pd.read_csv(file)
            # Убираем пробелы в названиях столбцов (если есть)
            df.columns = df.columns.str.strip()
            df_list.append(df)
        except Exception as e:
            print(f"Ошибка при чтении файла {file}: {e}")
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        return merged_df
    else:
        print("Не найдено CSV‑файлов в указанной папке.")
        return pd.DataFrame()

def sample_balanced(df, sample_fraction=0.02, label_col="Label", target_value="BENIGN"):
    """
    Отбирает случайным образом sample_fraction (в данном случае 2%) строк из DataFrame, 
    причём половина из этих строк должна иметь label_col == target_value (BENIGN),
    а вторая половина — остальные значения метки.
    
    Если строк в одной из групп не хватает, итоговый набор будет меньше, 
    но баланс 50/50 сохранится. При этом каждая выбранная строка 
    сохраняется целиком (все столбцы).
    """
    total_rows = len(df)
    # Общее число строк, которое хотим оставить
    desired_total = int(total_rows * sample_fraction)
    # Половина от этого числа
    desired_half = desired_total // 2

    # Делим на две группы
    df_target = df[df[label_col] == target_value]      # BENIGN
    df_other = df[df[label_col] != target_value]       # всё остальное

    # Сколько строк реально доступно в каждой группе
    available_target = len(df_target)
    available_other = len(df_other)

    # Берём максимум из того, что можем (иначе может не хватить строк)
    half_count = min(desired_half, available_target, available_other)
    if half_count < desired_half:
        print("Внимание: строк в одной из групп меньше, чем нужно для выборки. Итоговый датасет будет меньше.")

    # Случайным образом выбираем нужное количество строк из обеих групп
    sampled_target = df_target.sample(n=half_count, random_state=42)
    sampled_other = df_other.sample(n=half_count, random_state=42)
    
    # Объединяем и перемешиваем
    sampled_df = pd.concat([sampled_target, sampled_other], ignore_index=True)
    sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    return sampled_df

# ================================
# Новая часть: функция для формирования второй выборки
# ================================
def sample_balanced_second(merged_df, first_sample, sample_fraction=0.02, label_col="Label", target_value="BENIGN", overlap_ratio=0.1):
    """
    Формирует вторую выборку размером sample_fraction от всего датасета, сбалансированную по метке.
    При этом гарантируется, что примерно overlap_ratio (10%) строк будут взяты из первой выборки (first_sample),
    а оставшиеся (90%) – из оставшихся строк (merged_df без строк из first_sample).

    Алгоритм:
      1. Определяем требуемый размер второй выборки (n2).
      2. Вычисляем, сколько строк overlap_count (из first_sample) и сколько new_count (из оставшихся).
      3. Для балансировки делим каждую выборку на строки с target_value и остальные.
      4. Формируем overlap-часть и новую часть, после чего объединяем их.
    """
    total_rows = len(merged_df)
    n2 = int(total_rows * sample_fraction)
    if n2 == 0:
        print("Слишком маленький датасет для формирования второй выборки.")
        return pd.DataFrame()
        
    overlap_count = int(n2 * overlap_ratio)
    new_count = n2 - overlap_count

    # Целевое число строк с target_value и остальных во второй выборке
    target_total = n2 // 2
    other_total = n2 - target_total

    # Из overlap части делим overlap_count поровну:
    overlap_target = overlap_count // 2
    overlap_other = overlap_count - overlap_target

    # Для новой части, которые нужно добавить:
    new_target = target_total - overlap_target
    new_other = other_total - overlap_other

    # Из первой выборки берем overlap часть
    first_target = first_sample[first_sample[label_col] == target_value]
    first_other = first_sample[first_sample[label_col] != target_value]

    if len(first_target) < overlap_target or len(first_other) < overlap_other:
        print("Внимание: в первой выборке недостаточно строк для формирования overlap части.")
        overlap_target = min(overlap_target, len(first_target))
        overlap_other = min(overlap_other, len(first_other))
    
    overlap_sample_target = first_target.sample(n=overlap_target, random_state=101)
    overlap_sample_other = first_other.sample(n=overlap_other, random_state=101)
    overlap_sample = pd.concat([overlap_sample_target, overlap_sample_other], ignore_index=True)
    
    # Новая часть: из merged_df, исключая строки из первой выборки
    remaining_df = merged_df.drop(first_sample.index)
    new_target_pool = remaining_df[remaining_df[label_col] == target_value]
    new_other_pool = remaining_df[remaining_df[label_col] != target_value]
    
    if len(new_target_pool) < new_target or len(new_other_pool) < new_other:
        print("Внимание: недостаточно строк в оставшихся данных для формирования новой части второй выборки.")
        new_target = min(new_target, len(new_target_pool))
        new_other = min(new_other, len(new_other_pool))
    
    new_sample_target = new_target_pool.sample(n=new_target, random_state=102)
    new_sample_other = new_other_pool.sample(n=new_other, random_state=102)
    new_sample = pd.concat([new_sample_target, new_sample_other], ignore_index=True)
    
    # Объединяем overlap и новую часть, перемешиваем
    second_sample = pd.concat([overlap_sample, new_sample], ignore_index=True)
    second_sample = second_sample.sample(frac=1, random_state=103).reset_index(drop=True)
    return second_sample
# ================================
# Конец новой части
# ================================

def main():
    # Путь к папке с CSV‑файлами
    folder = input("Введите путь к папке с CSV‑файлами: ").strip()
    if not os.path.isdir(folder):
        print("Указанная папка не существует.")
        return

    # 1) Объединяем все CSV‑файлы
    merged_df = merge_csv_files(folder)
    if merged_df.empty:
        return
    print(f"Объединено строк: {len(merged_df)}")
    print("Найденные столбцы:", merged_df.columns.tolist())

    # Проверяем, есть ли столбец 'Label'
    if "Label" not in merged_df.columns:
        print("Столбец 'Label' не найден в данных!")
        return

    # (Необязательно) Посмотрим, какие уникальные значения встречаются в метках
    print("Уникальные значения в столбце 'Label':", merged_df["Label"].unique())
    print("Количество строк по типам меток:\n", merged_df["Label"].value_counts())

    # 2) Формируем сбалансированную выборку 2% (как в вашем коде)
    sampled_df = sample_balanced(merged_df, sample_fraction=0.02, label_col="Label", target_value="BENIGN")
    print(f"Размер итогового набора (примерно 2%): {len(sampled_df)}")

    # 3) Сохраняем первую выборку (все столбцы, включая Label)
    output_file = r"ML&TEST\merged_sampled.csv"
    sampled_df.to_csv(output_file, index=False)
    print(f"Файл c выборкой сохранён: {output_file}")

    # 4) По желанию — создаём второй файл с другой выборкой строк,
    #    из которой удаляем столбец 'Label'
    create_second_variant = input("Создать второй файл с другой выборкой (и без столбца 'Label')? (да/нет): ").strip().lower()
    if create_second_variant in ["да", "yes"]:
        second_sample = sample_balanced_second(merged_df, sampled_df, sample_fraction=0.02, label_col="Label", target_value="BENIGN", overlap_ratio=0.1)
        if not second_sample.empty:
            # Удаляем столбец 'Label' из второй выборки
            second_sample_no_label = second_sample.drop(columns=["Label"])
            output_file2 = r"ML&TEST\merged_sampled_no_labels.csv"
            second_sample_no_label.to_csv(output_file2, index=False)
            print(f"Второй файл (новая выборка без столбца 'Label') сохранён: {output_file2}")

if __name__ == "__main__":
    main()


Введите путь к папке с CSV‑файлами:  C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\Обучение\Уменьшенный датасет ML\MachineLearningCVE


Объединено строк: 2830743
Найденные столбцы: ['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG F

Создать второй файл с другой выборкой (и без столбца 'Label')? (да/нет):  да


Второй файл (новая выборка без столбца 'Label') сохранён: ML&TEST\merged_sampled_no_labels.csv
